# **Point Scanning Super Resolution (PSSR) - Training**

# **1. Preparation: Set the Runtime type and mount your Google Drive**
---

## **1.1. Set the Runtime type**
---

<font size = 4>Go to **Runtime -> Change the Runtime type**

<font size = 4>**Runtime type: Python 3** *(Python 3 is programming language in which this program is written)*

<font size = 4>**Accelator: GPU** *(Graphics processing unit (GPU)*

In [1]:
#Run this cell to check if you have GPU access

import tensorflow as tf
if tf.test.gpu_device_name()=='':
  print('You do not have GPU access.') 
  print('Did you change your runtime ?') 
  print('If the runtime settings are correct then Google did not allocate GPU to your session')
  print('Expect slow performance. To access GPU try reconnecting later')
else:
  print('You have GPU access')

from tensorflow.python.client import device_lib 
device_lib.list_local_devices()

2023-02-21 16:47:12.202774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


You do not have GPU access.
Did you change your runtime ?
If the runtime settings are correct then Google did not allocate GPU to your session
Expect slow performance. To access GPU try reconnecting later


2023-02-21 16:47:13.428598: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16091977336307693434
 xla_global_id: -1]

## **1.2. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive. 

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

## **1.3. Install PSSR and dependencies**
---

## **1.4. Specify your working folder - need your input**
---

In [3]:
root_path = "/content/gdrive/MyDrive/PSSR-32bit"

# **2. PSSR - Get to know your training data**
---

In [2]:
import sys
sys.path.insert(1, root_path)
from fastai import *
from fastai.vision import *
from fastcore.script import *
from utils import *
from pathlib import Path
from fastprogress import master_bar, progress_bar
from time import sleep
import shutil
import PIL
import czifile
PIL.Image.MAX_IMAGE_PIXELS = 99999999999999

NameError: name 'root_path' is not defined

## **2.1. Specify your datasource - need your input**
---

In [ ]:
sources = [root_path + '/32bitTS'] #must be in form of array
output_file = 'mod_test.csv'
only = 'mod_test'
skip = ''

In [ ]:
src_dirs = []
for src in sources:
    sub_fldrs = subfolders(Path(src))
    if skip: src_dirs += [fldr for fldr in sub_fldrs if fldr.stem not in skip]
    elif only: src_dirs += [fldr for fldr in sub_fldrs if fldr.stem in only]
    else: src_dirs += sub_fldrs

In [ ]:
#@title process_czi()
def process_czi(item, category, mode):
#This function only takes the first channel of the czi files
#since those are the only mitotracker channels
    tif_srcs = []
    base_name = item.stem
    with czifile.CziFile(item) as czi_f:
        data = czi_f.asarray()
        axes, shape = get_czi_shape_info(czi_f)
        channels = shape['C']
        depths = shape['Z']
        times = shape['T']
        #times = min(times, 30) #ONLY USE FIRST 30 frames
        x,y = shape['X'], shape['Y']

        mid_depth = depths // 2
        depth_range = range(max(0,mid_depth-2), min(depths, mid_depth+2))
        is_multi = (times > 1) or (depths > 1)

        data = czi_f.asarray()
        all_rmax = data.max()
        all_mi, all_ma = np.percentile(data, [2,99.99])

        dtype = data.dtype
        #for channel in range(channels): #if other channels are needed, use this line
        for channel in range(0,1):
            for z in depth_range:
                for t in range(times):
                    idx = build_index(
                        axes, {
                            'T': t,
                            'C': channel,
                            'Z': z,
                            'X': slice(0, x),
                            'Y': slice(0, y)
                        })
                    img = data[idx]
                    mi, ma = np.percentile(img, [2,99.99])
                    if dtype == np.uint8: rmax = 255.
                    else: rmax = img.max()
                    tif_srcs.append({'fn': item, 'ftype': 'czi', 'multi':int(is_multi), 'category': category, 'dsplit': mode,
                                     'uint8': dtype == np.uint8, 'mi': mi, 'ma': ma, 'rmax': rmax,
                                     'all_rmax': all_rmax, 'all_mi': all_mi, 'all_ma': all_ma,
                                     'mean': img.mean(), 'sd': img.std(),
                                     'nc': channels, 'nz': depths, 'nt': times,
                                     'z': z, 't': t, 'c':channel, 'x': x, 'y': y})
    return tif_srcs

In [ ]:
def is_live(item):
    return item.parent.parts[-3] == 'live'

def process_tif(item, category, mode):
    tif_srcs = []
    img = PIL.Image.open(item)
    n_frames = img.n_frames
    x,y = img.size
    is_multi = n_frames > 1
    #n_frames = min(n_frames, 30) #ONLY USE FIRST 30 frames

    data = []
    for n in range(n_frames):
        img.seek(n)
        img.load()
        img_data = np.array(img)
        data.append(img_data)

    data = np.stack(data)
    all_rmax = data.max()
    all_mi, all_ma = np.percentile(data, [2,99.99])

    for n in range(n_frames):
        img_data = data[n]
        dtype = img_data.dtype
        mi, ma = np.percentile(img_data, [2,99.99])
        if dtype == np.uint8: rmax = 255.
        else: rmax = img_data.max()
        if is_live(item):
            t, z = n, 0
            nt, nz = n_frames, 1
        else:
            t, z = 0, n
            nt, nz = 1, n_frames
        
        print(item)

        tif_srcs.append({'fn': item, 'ftype': 'tif', 'multi':int(is_multi), 'category': category, 'dsplit': mode,
                         'uint8': dtype==np.uint8, 'mi': mi, 'ma': ma, 'rmax': rmax,
                         'all_rmax': all_rmax, 'all_mi': all_mi, 'all_ma': all_ma,
                         'mean': img_data.mean(), 'sd': img_data.std(),
                         'nc': 1, 'nz': nz, 'nt': nt,
                         'z': z, 't': t, 'c':0, 'x': x, 'y': y})
    return tif_srcs

def process_unk(item, category, mode):
    print(f"**** Unknown: {item}")
    return []

def process_item(item, category, mode):
    try:
        if mode == 'test': return []
        else:
            item_map = {
                '.tif': process_tif,
                '.tiff': process_tif,
                '.czi': process_czi,
            }
            map_f = item_map.get(item.suffix, process_unk)
            return map_f(item, category, mode)
    except Exception as ex:
        print(f'err procesing: {item}')
        print(ex)
        return []

def build_tifs(src, mbar=None):
    tif_srcs = []
    for mode in ['train', 'valid', 'test']:
        live = src.parent.parts[-1] == 'live'
        src_dir = src / mode
        category = src.stem
        items = list(src_dir.iterdir()) if src_dir.exists() else []
        if items:
            for p in progress_bar(items, parent=mbar):
                mbar.child.comment = mode
                tif_srcs += process_item(p, category=category, mode=mode)
    return tif_srcs

In [ ]:
#pull metadata from datasources
mbar = master_bar(src_dirs)
tif_srcs = []
for src in mbar:
    mbar.write(f'process {src.stem}')
    tif_srcs += build_tifs(src, mbar=mbar)

process mod_test
/content/gdrive/MyDrive/PSSR-32bit/32bitTS/mod_test/train/TS0006.tif
/content/gdrive/MyDrive/PSSR-32bit/32bitTS/mod_test/valid/TS0014.tif


In [ ]:
#save csv to disk
tif_src_df = pd.DataFrame(tif_srcs)
tif_src_df[['category','dsplit','multi','ftype','uint8','mean','sd','all_rmax','all_mi','all_ma','mi','ma','rmax','nc','nz','nt','c','z','t','x','y','fn']].to_csv(output_file, header=True, index=False)
shutil.move(output_file, f'{root_path}/{output_file}')


'/content/gdrive/MyDrive/PSSR-32bit/mod_test.csv'